In [ ]:
from models_impute import *
from pygrinder import block_missing

In [ ]:
data = pd.read_csv("./data/lorenz/lorenz_dataset_0_timeseries.csv",header=None)
data = data.values

In [ ]:
cg = pd.read_csv('./causality_matrices/lorenz_causality_matrix.csv', header=None)
cg = cg.values
model_params = {
        'num_levels': 10,
        'kernel_size': 8,
        'dilation_c': 2
    }

In [ ]:
data = block_missing(data[np.newaxis,...],factor=0.1, block_len=3, block_width=3)
data = data[0]
data

In [ ]:
data_imputed = impute(data, cg, model_params)
data_imputed

In [ ]:
# test_miracle.py
import numpy as np
from baseline import miracle_impu

# 创建测试数据
test_data = np.random.randn(100, 10)
test_data[np.random.random((100, 10)) < 0.1] = np.nan

print("测试数据形状:", test_data.shape)
print("缺失值数量:", np.isnan(test_data).sum())

try:
    result = miracle_impu(test_data)
    print("MIRACLE结果形状:", result.shape if result is not None else "None")
    print("MIRACLE结果类型:", type(result))
    
    if result is not None:
        print("填补后缺失值:", np.isnan(result).sum())
    else:
        print("❌ MIRACLE返回了None")
        
except Exception as e:
    print(f"❌ MIRACLE测试失败: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
zero_matrix = pd.read_csv("zero_impu_matrix.csv").values  # shape: (T, N)

# 构造掩码矩阵：0 的位置为 0，其余为 1
mask_matrix = np.where(zero_matrix == 0, 0, 1)

# 保存新矩阵到 CSV
pd.DataFrame(mask_matrix).to_csv("zero_impu_mask.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pred = pd.read_csv('./train_result.csv').values
gt = pd.read_csv('./gt_matrix.csv').values

In [ ]:
import torch.nn.functional as F
import torch

In [ ]:
mask_matrix = 1 - mask_matrix

In [ ]:
pred*mask_matrix

In [ ]:
gt*mask_matrix

In [ ]:
res = F.mse_loss(torch.tensor(pred*mask_matrix), torch.tensor(gt*mask_matrix))
res

In [ ]:
import tempfile
import os

tmp_file = tempfile.NamedTemporaryFile(delete=False)
tmp_file.close()
os.remove(tmp_file.name)


In [ ]:
import os
import shutil

# 原始目录和目标目录路径
src_dir = './data/III'        # 替换为你的源目录路径
dst_dir = './data/mimic-iii'   # 替换为你的目标目录路径

# 创建目标目录（如果不存在）
os.makedirs(dst_dir, exist_ok=True)

# 获取源目录中的所有文件名（按名称排序，可改为按修改时间等）
all_files = sorted(f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)))

# 选择前100个文件
files_to_move = all_files[:100]

# 移动文件
for fname in files_to_move:
    shutil.move(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))

print(f"已成功移动 {len(files_to_move)} 个文件到 {dst_dir}")


In [1]:
import os
import pandas as pd
import numpy as np
from baseline import *
from models_impute import *
# 指定目录
data_dir = "./data/III"  # 替换为你的目录路径

# 获取目录下第一个 CSV 文件
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]
assert csv_files, "目录中没有找到 CSV 文件"
csv_path = os.path.join(data_dir, csv_files[0])

# 读取数据
df = pd.read_csv(csv_path)
mx = df.values.astype(np.float32)

# 记录原始缺失位置
nan_pos = np.argwhere(np.isnan(mx))

# 填补缺失值
imputed_mx = timesnet_impu(mx,)

# 打印填补前后的值（仅缺失位置）
print(f"填补前后对比（仅缺失位置）：{imputed_mx}")



2025-07-21 23:29:49 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 

填补前后对比（仅缺失位置）：[[ 93.       137.        64.       ...  60.061043  60.061043   0.      ]
 [  0.         0.         0.       ...  60.061043  60.061043   7.      ]
 [  0.         0.         0.       ...  60.061043  60.061043   0.      ]
 ...
 [  0.         0.         0.       ...  60.061043  60.061043   0.      ]
 [  0.         0.         0.       ...  60.0

In [ ]:
import pkgutil
import sys

for _, name, _ in pkgutil.iter_modules():
    if 'tsde' in name:
        print(name)


In [26]:
import os
import numpy as np
import pandas as pd
from pygrinder import mcar, mar_logistic, mnar_x

def dig_missing_values(
    file_path: str,
    output_dir: str,
    obs_rate: float = 0.4,
    mar_missing_rate: float = 0.6,
    mnar_offset: float = 0.7,
    mcar_p: float = 0.1
):
    # 读取数据
    df = pd.read_csv(file_path)
    data = df.values.astype(np.float32)
    
    # 应用缺失机制
    X = mar_logistic(data, obs_rate=obs_rate, missing_rate=mar_missing_rate)
    X = X[np.newaxis, ...]                  # 添加 batch 维度
    X = mnar_x(X, offset=mnar_offset)       # MNAR 缺失
    X = mcar(X, p=mcar_p)                   # 再次添加 MCAR 缺失
    X = X.squeeze(0)                        # 去除 batch 维度

    # 保存结果
    filename = os.path.basename(file_path).replace('.csv', '_dig_missing.csv')
    output_path = os.path.join(output_dir, filename)
    pd.DataFrame(X).to_csv(output_path, index=False)
    print(f"✅ 保存至: {output_path}")
dig_missing_values(
    file_path='./data/finance/finance_dataset_0_timeseries.csv',
    output_dir='./',)

✅ 保存至: ./finance_dataset_0_timeseries_dig_missing.csv


In [ ]:
import tsdb

# list all available datasets in TSDB
tsdb.list()
# ['physionet_2012',
#  'physionet_2019',
#  'electricity_load_diagrams',
#  'beijing_multisite_air_quality',
#  'italy_air_quality',
#  'vessel_ais',
#  'electricity_transformer_temperature',
#  'pems_traffic',
#  'solar_alabama',
#  'ucr_uea_ACSF1',
#  'ucr_uea_Adiac',
#  ...

tsdb.download_and_extract('beijing_multisite_air_quality', './save_it_here')
